In [9]:
import psycopg2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Dictionnaire de correspondance entre les noms des partis politiques et leurs identifiants numériques
id_to_party_dict = {
    1: "Lutte Ouvrière",
    2: "Parti Communiste Français",
    3: "La République En Marche",
    4: "Résistons",
    5: "Rassemblement National",
    6: "Reconquête",
    7: "La France Insoumise",
    8: "Parti Socialiste",
    9: "Europe Écologie Les Verts",
    10: "Les Républicains",
    11: "Nouveau Parti Anticapitaliste",
    12: "Debout La France"
}

# Paramètres de connexion à la base de données PostgreSQL
conn_params = {
    "host": "localhost",
    "port": "15432", 
    "database": "postgres",
    "user": "admin",
    "password": "admin"
}

# Connexion à la base de données
conn = psycopg2.connect(**conn_params)

# Requête SQL pour sélectionner les données de la table election_2022_t1
query_election = """
    SELECT winner, code_postal, "Libellé de la commune"
    FROM election_2022_t1
"""
df_election = pd.read_sql(query_election, conn)

# Fermeture de la connexion
conn.close()

# Mapper les noms des partis politiques aux identifiants numériques
party_to_id_dict = {v: k for k, v in id_to_party_dict.items()}
df_election["winner"] = df_election["winner"].map(party_to_id_dict)

# Remplacer les valeurs manquantes dans la colonne "winner" par une valeur arbitraire
df_election["winner"].fillna(-1, inplace=True)

# Convertir la colonne "winner" en float
df_election["winner"] = df_election["winner"].astype(float)

# Encodage des variables catégorielles
label_encoder = LabelEncoder()

# Exclure les données du département de l'Yonne (89) du jeu d'entraînement
df_train = df_election[df_election["code_postal"] != 89]

# Sélectionner uniquement les données du département de l'Yonne (89) pour les tests
df_test_yonne = df_election[df_election["code_postal"] == 89]

# Séparation des features et de la target pour l'ensemble d'entraînement
X_train = df_train.drop(columns=["Libellé de la commune", "winner"])
y_train = df_train["winner"]

# Séparation des features et de la target pour l'ensemble de test de l'Yonne
X_test_yonne = df_test_yonne.drop(columns=["Libellé de la commune", "winner"])
y_test_yonne = df_test_yonne["winner"]

# Entraînement du modèle de régression logistique multinomiale
model = LogisticRegression(max_iter=10000)
model.fit(X_train, y_train)

# Prédiction sur l'ensemble d'entraînement
y_pred_train = model.predict(X_train)

# Évaluation de la précision du modèle sur l'ensemble d'entraînement
accuracy_train = accuracy_score(y_train, y_pred_train)
print("Précision du modèle sur l'ensemble d'entraînement:", accuracy_train)

# Prédiction sur les données de test de l'Yonne
y_pred_yonne = model.predict(X_test_yonne)

# Évaluation de la précision du modèle sur les données de l'Yonne
accuracy_yonne = accuracy_score(y_test_yonne, y_pred_yonne)
print("Précision du modèle sur les données de l'Yonne:", accuracy_yonne)

# Afficher les prédictions du modèle sur les données de l'Yonne
print("Prédictions du modèle sur les données de l'Yonne:")
print(y_pred_yonne)


/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_9637/2051604164.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_election = pd.read_sql(query_election, conn)
/var/folders/9z/lnzs0dyx0k1g2nvs92hf0syr0000gn/T/ipykernel_9637/2051604164.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_election["winner"].fillna(-1, inplace=True)


Précision du modèle sur l'ensemble d'entraînement: 0.5734838149572119
Précision du modèle sur les données de l'Yonne: 0.7855072463768116
Prédictions du modèle sur les données de l'Yonne:
[5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5. 5.
 5. 5. 5. 